In [1]:
# General:
import tweepy           # To consume Twitter's API
import pandas as pd     # To handle data
import numpy as np      # For number computing

# For plotting and visualization:
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Twitter App access keys for @user

# Consume:
CONSUMER_KEY    = 'yZFeWEz9JjZleyUOlBGgnDXD6'
CONSUMER_SECRET = "v0VhWMfmOLbWujEmlpzttbrN0fvfNpC55zpo8laIpLF51tGyXs"

# Access:
ACCESS_TOKEN  = "385059774-Ndv0Azv3FaRVAw0Ibndu8KMdW5tdeoCBRZsh6ixW" 
ACCESS_SECRET = "qVq2j5tg3fdPE3eesKjnErPyiOt16BBaM0cToHgz0g8IR"


In [4]:
# We import our access keys:
# from credentials import *    # This will allow us to use the keys as variables

# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with our access keys provided.
    """
    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth)
    return api

In [5]:
# We create an extractor object:
extractor = twitter_setup()

# We create a tweet list as follows:
tweets = extractor.user_timeline(screen_name="realDonaldTrump", count=200)
print("Number of tweets extracted: {}.\n".format(len(tweets)))

# We print the most recent 5 tweets:
print("5 recent tweets:\n")
for tweet in tweets[:5]:
    print(tweet.text)
    print()

Number of tweets extracted: 200.

5 recent tweets:

Congressman Randy Hultgren (R) of Illinois is doing a great job. Get out and Vote for Randy - Total Endorsement!

Florida, very important - get out and vote for Florida Congressional Candidate Michael Waltz (R). He has my Strong Endorsement!

Epstein all the way in Michigan House 11. She is a wonderful person and, at the same time, a real fighter. Has my Strong Endorsement!

RT @FLOTUS: Vote Red 🇺🇸 MAGA 🇺🇸 https://t.co/i95OiBimxA https://t.co/ncvDcAP8LZ

Bob Hugin, successful all of his life, would be a Great Senator from New Jersey. He has my complete and total Endor… https://t.co/RBrSwuNZsh

